# LSA 
https://www.analyticsvidhya.com/blog/2018/10/stepwise-guide-topic-modeling-latent-semantic-analysis/


In [1]:
from utils import parse_xml
import string
import re
import nltk

stopwords = nltk.corpus.stopwords.words('english')

# Specify the path to your XML file
xml_file_path = r'C:\dev\NLP-Sandbox\PURE\requirements-xml\0000 - cctns.xml'
# Define the namespace
namespace = {'ns': 'req_document.xsd'}

# import utils.ParseXML as ParseXML
df = parse_xml.process_xml_with_namespace(xml_file_path, namespace)


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_colwidth", 200)

In [7]:
# removing everything except alphabets`
df['clean_doc'] = df['text'].str.replace("[^a-zA-Z#]", " ")

# removing short words
df['clean_doc'] = df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# make all text lowercase
df['clean_doc'] = df['clean_doc'].apply(lambda x: x.lower())

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# tokenization
tokenized_doc = df['clean_doc'].apply(lambda x: x.split())

# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

df['clean_doc'] = detokenized_doc

df.head(10)

,tag,text,id,path,clean_doc
0,title,E-GOVERNANCE MISSION MODE PROJECT (MMP),,req_document/title/title,e-governance mission mode project (mmp)
1,title,CRIME & CRIMINAL TRACKING NETWORK AND SYSTEMS (CCTNS),,req_document/title/title,crime criminal tracking network systems (cctns)
2,title,FUNCTIONAL REQUIREMENTS SPECIFICATION V1.0 (DRAFT),,req_document/title/title,functional requirements specification v1.0 (draft)
3,title,MINISTRY OF HOME AFFAIRS GOVERNMENT OF INDIA,,req_document/title/title,ministry home affairs government india
4,version,1.0,,req_document/version,
5,title,INTRODUCTION,1,req_document/p/title,introduction
6,title,The Functional Requirements Specifications (FRS) report provides the detailed description of the functionalities required for the first version of the CCTNS. The key guiding principle behind the f...,,req_document/p/text_body,functional requirements specifications (frs) report provides detailed description functionalities required first version cctns. guiding principle behind functional design cctns v1.0 focus critical...
7,title,FUNCTIONAL OVERVIEW,2,req_document/p/title,functional overview
8,title,"CCTNS V1.0 functionality is designed to focus on delivering value to IOs, records room staff and citizens within the broad crime investigation area. Based on the guiding principles stated above, n...",,req_document/p/text_body,"cctns v1.0 functionality designed focus delivering value ios, records room staff citizens within broad crime investigation area. based guiding principles stated above, nine different function bloc..."
9,title,DESCRIPTION OF THE MODULES AND FUNCTIONAL REQUIREMENTS,3,req_document/p/title,description modules functional requirements


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # keep top 1000 terms 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(df['clean_doc'])

X.shape # check shape of the document-term matrix

from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

len(svd_model.components_)

terms = vectorizer.get_feature_names_out()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        #print("")

Topic 0: 
user
users
access
navigation
cases
application
groups
Topic 1: 
module
support
audit
access
functionality
police
data
Topic 2: 
user
allow
cases
users
groups
profiles
specified
Topic 3: 
solution
provide
application
interfaces
help
material
scenarios
Topic 4: 
support
module
interface
users
access
connections
navigation
Topic 5: 
functional
requirements
architecture
users
groups
description
overview
Topic 6: 
police
module
search
investigation
citizens
users
interface
Topic 7: 
search
support
navigation
connections
multiple
list
include
Topic 8: 
navigation
components
functional
audit
trail
help
access
Topic 9: 
functionality
interface
downtime
exceed
month
period
rolling
Topic 10: 
downtime
exceed
month
period
rolling
unplanned
module
Topic 11: 
functionality
text
user
interfaces
search
access
data
Topic 12: 
text
navigation
cases
able
display
provide
9241
Topic 13: 
page
access
interfaces
designed
data
pages
network
Topic 14: 
windows
links
opening
browser
open
clearly
pop


In [23]:
import umap.umap_ as umap


X_topics = svd_model.fit_transform(X)
embedding = umap.UMAP(n_neighbors=150, min_dist=0.5, random_state=12).fit_transform(X_topics)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], 
c = dataset.target,
s = 10, # size
edgecolor='none'
)
plt.show()

c:\ProgramData\anaconda3\Lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


NameError: name 'dataset' is not defined

<Figure size 700x500 with 0 Axes>